# Importing libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# General Libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go

# Algorithms (Regression)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as XGBoost

# Preprocessing / Feature Selection / Model Selection / Metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from multiprocessing import Pool, Process
import random

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

pd.options.display.max_columns = 200

# Functions

In [3]:
def quality_metrics(y_true: float, y_pred: float) -> dict:
  dict_metrics = dict()
  dict_metrics['MSE'] = mean_squared_error(y_true, y_pred)
  dict_metrics['MAE'] = mean_absolute_error(y_true, y_pred)
  dict_metrics['RMSE'] = np.sqrt(dict_metrics['MSE'])
  return dict_metrics

# Importing datasets

In [4]:
#Real WP6++ dataset
real_df = pd.read_csv("/content/drive/MyDrive/Files For Project/AtomPairsData.csv")
real_df = real_df.drop(["Unnamed: 0"], axis = 1)
#Synthetic WP6++ dataset
gen_df = pd.read_csv("/content/drive/MyDrive/Files For Project/GeneratedData.csv")
gen_df = gen_df.drop(["Unnamed: 0"], axis = 1)

print(f"Length of real dataset: {real_df.shape[0]}")
print(f"Length of synthetic dataset: {gen_df.shape[0]}")

Length of real dataset: 21
Length of synthetic dataset: 31


There will be 3 datasets:

* 7 synthetic samples + 21 real samples (25% of synthetic data)  
* 14 synthetic + 21 real (50%)
* 31 syntheric + 21 real (60%)



# Ratio of 7:21 (25%)

In [5]:
adding = gen_df.head(7)
df = pd.concat([real_df, adding], ignore_index=True)
df.shape

(28, 13)

In [6]:
X1 = df.drop(['Ka_mean'], axis=1)
y1 = df["Ka_mean"]
print(f'General quantity of samples: {X1.shape[0]}')
print(f'Quantity of features: {X1.shape[1]}')

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.1, shuffle=False, random_state=42)
print(f"Length of training data: {len(X_train1)}")
print(f"Length of testing data: {len(X_test1)}")

General quantity of samples: 28
Quantity of features: 12
Length of training data: 25
Length of testing data: 3


## Building machine learning models

### Linear Regression

In [7]:
# Creating and fitting model
Lin_regressor1 = LinearRegression(n_jobs=-1)
grid_search_cv_linear1 = GridSearchCV(Lin_regressor1, {}, cv=5)
grid_search_cv_linear1.fit(X_train1, y_train1.to_numpy())

# Model predictions
y_pred_Linear_train1 = grid_search_cv_linear1.best_estimator_.predict(X_train1)
y_pred_Linear_test1 = grid_search_cv_linear1.best_estimator_.predict(X_test1)

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train1, y_pred=y_pred_Linear_train1).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
lin_reg_metrics1 = quality_metrics(y_true=y_test1, y_pred=y_pred_Linear_test1)
for name_metric, error in lin_reg_metrics1.items():
    print(f'Test {name_metric}: {error:.5f}')

Train MSE: 0.02666
Train MAE: 0.09439
Train RMSE: 0.16328

Test MSE: 0.03082
Test MAE: 0.13329
Test RMSE: 0.17556


### Ridge Regression

In [8]:
# Creating and fitting model
Ridge_regressor1 = Ridge()
ridge_reg_parameters1 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_ridge1 = GridSearchCV(Ridge_regressor1, ridge_reg_parameters1, cv=5)

grid_search_cv_ridge1.fit(X_train1, y_train1)
grid_search_cv_ridge1.best_params_

# Model predictions
Ridge_best_reg1 = grid_search_cv_ridge1.best_estimator_
y_pred_Ridge_train1 = Ridge_best_reg1.predict(X_train1)
y_pred_Ridge_test1 = Ridge_best_reg1.predict(X_test1)

#The best Hyperparameters
ridge_best_key1 = list((grid_search_cv_ridge1.best_params_).keys())[0]
ridge_best_value1 = list((grid_search_cv_ridge1.best_params_).values())[0]
print(f'The best hyperparameters: {ridge_best_key1} = {ridge_best_value1}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train1, y_pred=y_pred_Ridge_train1).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
ridge_metrics1 = quality_metrics(y_true=y_test1, y_pred=y_pred_Ridge_test1)
for name_metric, error in ridge_metrics1.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 10.0

Train MSE: 0.03965
Train MAE: 0.12373
Train RMSE: 0.19913

Test MSE: 0.00493
Test MAE: 0.04928
Test RMSE: 0.07024


### Lasso Regresssion

In [9]:
# Creating and fitting model
Lasso_regressor1 = Lasso()
lasso_reg_parameters1 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_lasso1 = GridSearchCV(Lasso_regressor1, lasso_reg_parameters1, cv=5)

grid_search_cv_lasso1.fit(X_train1, y_train1)
grid_search_cv_lasso1.best_params_

# Model predictions
Lasso_best_reg1 = grid_search_cv_lasso1.best_estimator_
y_pred_Lasso_train1 = Lasso_best_reg1.predict(X_train1)
y_pred_Lasso_test1 = Lasso_best_reg1.predict(X_test1)

#The best Hyperparameters
lasso_best_key1 = list((grid_search_cv_lasso1.best_params_).keys())[0]
lasso_best_value1 = list((grid_search_cv_lasso1.best_params_).values())[0]
print(f'The best hyperparameters: {lasso_best_key1} = {lasso_best_value1}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train1, y_pred=y_pred_Lasso_train1).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
lasso_metrics1 = quality_metrics(y_true=y_test1, y_pred=y_pred_Lasso_test1)
for name_metric, error in lasso_metrics1.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 0.18329807108324375

Train MSE: 0.06744
Train MAE: 0.16594
Train RMSE: 0.25969

Test MSE: 0.00535
Test MAE: 0.06892
Test RMSE: 0.07315


### ElasticNet Regression

In [10]:
# Creating and fitting model
ElasticNet_regressor1 = ElasticNet()
elasticnet_reg_parameters1 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_elasticnet1 = GridSearchCV(ElasticNet_regressor1, elasticnet_reg_parameters1, cv=5)

grid_search_cv_elasticnet1.fit(X_train1, y_train1)
grid_search_cv_elasticnet1.best_params_

# Model predictions
Elasticnet_best_reg1 = grid_search_cv_elasticnet1.best_estimator_
y_pred_Elasticnet_train1 = Elasticnet_best_reg1.predict(X_train1)
y_pred_Elasticnet_test1 = Elasticnet_best_reg1.predict(X_test1)

#The best Hyperparameters
elasticnet_best_key1 = list((grid_search_cv_elasticnet1.best_params_).keys())[0]
elasticnet_best_value1 = list((grid_search_cv_elasticnet1.best_params_).values())[0]
print(f'The best hyperparameters: {elasticnet_best_key1} = {elasticnet_best_value1}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train1, y_pred=y_pred_Elasticnet_train1).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
elasticnet_metrics1 = quality_metrics(y_true=y_test1, y_pred=y_pred_Elasticnet_test1)
for name_metric, error in elasticnet_metrics1.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 0.04832930238571752

Train MSE: 0.04243
Train MAE: 0.12839
Train RMSE: 0.20600

Test MSE: 0.00398
Test MAE: 0.05391
Test RMSE: 0.06309


### Random Forest

In [11]:
# Creating and fitting model
Forest_regressor1 = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)
forest_reg_parameters1 = {'max_depth': np.arange(4, 20, 2)}
grid_search_cv_forest1 = GridSearchCV(Forest_regressor1, forest_reg_parameters1, cv=3)
grid_search_cv_forest1.fit(X_train1, y_train1)
grid_search_cv_forest1.best_params_

# Model predictions
Forest_best_reg1 = grid_search_cv_forest1.best_estimator_
y_pred_Forest_train1 = Forest_best_reg1.predict(X_train1)
y_pred_Forest_test1 = Forest_best_reg1.predict(X_test1)

#The best Hyperparameters
forest_best_key1 = list((grid_search_cv_forest1.best_params_).keys())[0]
forest_best_value1 = list((grid_search_cv_forest1.best_params_).values())[0]
print(f'The best hyperparameters: {forest_best_key1} = {forest_best_value1}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train1, y_pred=y_pred_Forest_train1).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
rforest_metrics1 = quality_metrics(y_true=y_test1, y_pred=y_pred_Forest_test1)
for name_metric, error in rforest_metrics1.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: max_depth = 6

Train MSE: 0.02380
Train MAE: 0.07971
Train RMSE: 0.15426

Test MSE: 0.00126
Test MAE: 0.03053
Test RMSE: 0.03552


### k-NN Regression

In [12]:
# Creating and fitting model
KNeighbors_regressor1 = KNeighborsRegressor()
kNN_reg_parameters1 = {'n_neighbors': np.arange(4, 20, 2)}                  # <- np.arange(2, 6, 1)
grid_search_cv_kNN1 = GridSearchCV(KNeighbors_regressor1, kNN_reg_parameters1, cv=5)
grid_search_cv_kNN1.fit(X_train1, y_train1)
grid_search_cv_kNN1.best_params_

# Model predictions
kNN_best_reg1 = grid_search_cv_kNN1.best_estimator_
y_pred_kNN_train1 = kNN_best_reg1.predict(X_train1)
y_pred_kNN_test1 = kNN_best_reg1.predict(X_test1)

#The best Hyperparameters
kNN_best_key1 = list((grid_search_cv_kNN1.best_params_).keys())[0]
kNN_best_value1 = list((grid_search_cv_kNN1.best_params_).values())[0]
print(f'The best hyperparameters: {kNN_best_key1} = {kNN_best_value1}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train1, y_pred=y_pred_kNN_train1).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
knn_metrics1 = quality_metrics(y_true=y_test1, y_pred=y_pred_kNN_test1)
for name_metric, error in knn_metrics1.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: n_neighbors = 12

Train MSE: 0.04723
Train MAE: 0.12069
Train RMSE: 0.21732

Test MSE: 0.00150
Test MAE: 0.03507
Test RMSE: 0.03870


### Boosting

In [13]:
np.random.seed(42)
random.seed(42)

# Creating and fitting model
GBoosting_regressor1 = GradientBoostingRegressor(n_estimators=50, max_depth=18)
gboost_reg_parameters1 = {}
grid_search_cv_gboost1 = GridSearchCV(GBoosting_regressor1, gboost_reg_parameters1, cv=5)
grid_search_cv_gboost1.fit(X_train1, y_train1)
#print(grid_search_cv_gboost.best_params_)

# Model predictions
GBoost_best_reg1 = grid_search_cv_gboost1.best_estimator_
y_pred_gboost_train1 = GBoost_best_reg1.predict(X_train1)
y_pred_gboost_test1 = GBoost_best_reg1.predict(X_test1)

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train1, y_pred=y_pred_gboost_train1).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
gboost_metrics1 = quality_metrics(y_true=y_test1, y_pred=y_pred_gboost_test1)
for name_metric, error in gboost_metrics1.items():
    print(f'Test {name_metric}: {error:.5f}')

Train MSE: 0.01938
Train MAE: 0.04003
Train RMSE: 0.13922

Test MSE: 0.00185
Test MAE: 0.03257
Test RMSE: 0.04303


## Evaluation

In [14]:
fig = go.Figure()

model_name = ['Ridge', "Lasso","ElasticNet", 'RForest', 'kNN', 'Gradient Boosting']
model_metrics = [ridge_metrics1,lasso_metrics1, elasticnet_metrics1, rforest_metrics1, knn_metrics1, gboost_metrics1]

for name, metrics in zip(model_name, model_metrics):
    fig.add_trace(go.Histogram(histfunc= 'max',
                            x = list(metrics.keys()),
                                y = list(metrics.values()),
                                name = name,
                                ))

fig.update_layout(font_size = 25,
                    font_color='black',
                    title = "Model Evaluation",
                    plot_bgcolor = 'rgba(250,250,250,1)',
                    width = 1100,
                    height = 550,
                    # LEGEND
                    legend_title = "Models",
                    legend_font_size = 25,
                    legend_x = 1.02,
                    legend_y = 1,
                    #legend_bordercolor = 'black',
                    #legend_borderwidth = 1,
                    legend_itemsizing = 'trace',
                    legend_itemwidth=100,
                    # X-axis
                    xaxis_title = "WP6 + generated data (25%)",
                    xaxis_nticks = 7,
                    xaxis_ticklen = 16,
                    xaxis_tickwidth = 3,
                    xaxis_ticks = 'outside',
                    # Y-axis
                    yaxis_title = "Error",
                    yaxis_nticks = 10,
                    yaxis_ticklen = 16,
                    yaxis_tickwidth = 3,
                    yaxis_ticks = 'outside'
                    )

fig.add_shape(type="rect",
                xref="paper",
                yref="paper",
                x0=0,
                y0=0,
                x1=1.0,
                y1=1.0,
        line=dict(
            color="black",
                width=1,))
fig.layout.font.family = 'sans-serif'

fig.show()

> The metrics are already **much** better than just after using the standard WP6 dataset!

### Table of metrics

In [15]:
#Metrics table
data = []

for i in range(3):
    row = []
    for j in range(6):
        key = list(model_metrics[j].keys())[i]
        row.append(model_metrics[j][key])
    data.append(row)

WP6_25_metrics = pd.DataFrame(data, columns=model_name[:len(data[0])])
WP6_25_metrics['Metric'] = ['MSE', 'MAE', 'RMSE']
WP6_25_metrics['Dataset'] = ['WP6 + gen.data (25%)', 'WP6 + gen.data (25%)', 'WP6 + gen.data (25%)']
WP6_25_metrics["Linear"] = list(lin_reg_metrics1.values())

WP6_25_metrics = WP6_25_metrics[['Metric', 'Dataset', 'Linear'] + WP6_25_metrics.columns[:-3].tolist()]
WP6_25_metrics

,Metric,Dataset,Linear,Ridge,Lasso,ElasticNet,RForest,kNN,Gradient Boosting
0,MSE,WP6 + gen.data (25%),0.030822,0.004933,0.005351,0.003981,0.001261,0.001498,0.001852
1,MAE,WP6 + gen.data (25%),0.133292,0.049281,0.068924,0.053907,0.030526,0.035074,0.032566
2,RMSE,WP6 + gen.data (25%),0.175563,0.070236,0.073148,0.063093,0.035517,0.038702,0.043029


### Table of hyperparameters

In [16]:
#Best hyperparameters table
hyperparams_dict1 = {"Name of ML model": ["Linear Regression", "Ridge", "Lasso", "ElasticNet",
                                         "Random Forest", "kNN Regression", "XGBoost"],
                    "Tuning Hyperparameter": ["-", "Alpha", "Alpha", "Alpha", "Max Depth", "Number of Neighbors", "Number of Estimators, Max Depth"],
                    "Value": [None, ridge_best_value1, lasso_best_value1, elasticnet_best_value1, forest_best_value1,
                              kNN_best_value1, [50, 18]]}

best_hyperparams1 = pd.DataFrame(hyperparams_dict1)
best_hyperparams1

,Name of ML model,Tuning Hyperparameter,Value
0,Linear Regression,-,None
1,Ridge,Alpha,10.0
2,Lasso,Alpha,0.183298
3,ElasticNet,Alpha,0.048329
4,Random Forest,Max Depth,6
5,kNN Regression,Number of Neighbors,12
6,XGBoost,"Number of Estimators, Max Depth","[50, 18]"


# Ratio of 14:21 (50%)

In [17]:
adding = gen_df.head(14)
df = pd.concat([real_df, adding], ignore_index=True)
df.shape

(35, 13)

In [18]:
X2 = df.drop(['Ka_mean'], axis=1)
y2 = df["Ka_mean"]
print(f'General quantity of samples: {X2.shape[0]}')
print(f'Quantity of features: {X2.shape[1]}')

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.1, shuffle=False, random_state=42)
print(f"Length of training data: {len(X_train2)}")
print(f"Length of testing data: {len(X_test2)}")

General quantity of samples: 35
Quantity of features: 12
Length of training data: 31
Length of testing data: 4


## Building machine learning models

### Linear Regression

In [19]:
# Creating and fitting model
Lin_regressor2 = LinearRegression(n_jobs=-1)
grid_search_cv_linear2 = GridSearchCV(Lin_regressor2, {}, cv=5)
grid_search_cv_linear2.fit(X_train2, y_train2.to_numpy())

# Model predictions
y_pred_Linear_train2 = grid_search_cv_linear2.best_estimator_.predict(X_train2)
y_pred_Linear_test2 = grid_search_cv_linear2.best_estimator_.predict(X_test2)

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train2, y_pred=y_pred_Linear_train2).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
lin_reg_metrics2 = quality_metrics(y_true=y_test2, y_pred=y_pred_Linear_test2)
for name_metric, error in lin_reg_metrics2.items():
    print(f'Test {name_metric}: {error:.5f}')

Train MSE: 0.02358
Train MAE: 0.09390
Train RMSE: 0.15356

Test MSE: 0.05829
Test MAE: 0.19480
Test RMSE: 0.24144


### Ridge Regression

In [20]:
# Creating and fitting model
Ridge_regressor2 = Ridge()
ridge_reg_parameters2 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_ridge2 = GridSearchCV(Ridge_regressor2, ridge_reg_parameters2, cv=5)

grid_search_cv_ridge2.fit(X_train2, y_train2)
grid_search_cv_ridge2.best_params_

# Model predictions
Ridge_best_reg2 = grid_search_cv_ridge2.best_estimator_
y_pred_Ridge_train2 = Ridge_best_reg2.predict(X_train2)
y_pred_Ridge_test2 = Ridge_best_reg2.predict(X_test2)

#The best Hyperparameters
ridge_best_key2 = list((grid_search_cv_ridge2.best_params_).keys())[0]
ridge_best_value2 = list((grid_search_cv_ridge2.best_params_).values())[0]
print(f'The best hyperparameters: {ridge_best_key2} = {ridge_best_value2}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train2, y_pred=y_pred_Ridge_train2).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
ridge_metrics2 = quality_metrics(y_true=y_test2, y_pred=y_pred_Ridge_test2)
for name_metric, error in ridge_metrics2.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 0.003359818286283788

Train MSE: 0.02359
Train MAE: 0.09394
Train RMSE: 0.15357

Test MSE: 0.05782
Test MAE: 0.19289
Test RMSE: 0.24045


### Lasso Regresssion

In [21]:
# Creating and fitting model
Lasso_regressor2 = Lasso()
lasso_reg_parameters2 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_lasso2 = GridSearchCV(Lasso_regressor2, lasso_reg_parameters2, cv=5)

grid_search_cv_lasso2.fit(X_train2, y_train2)
grid_search_cv_lasso2.best_params_

# Model predictions
Lasso_best_reg2 = grid_search_cv_lasso2.best_estimator_
y_pred_Lasso_train2 = Lasso_best_reg2.predict(X_train2)
y_pred_Lasso_test2 = Lasso_best_reg2.predict(X_test2)

#The best Hyperparameters
lasso_best_key2 = list((grid_search_cv_lasso2.best_params_).keys())[0]
lasso_best_value2 = list((grid_search_cv_lasso2.best_params_).values())[0]
print(f'The best hyperparameters: {lasso_best_key2} = {lasso_best_value2}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train2, y_pred=y_pred_Lasso_train2).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
lasso_metrics2 = quality_metrics(y_true=y_test2, y_pred=y_pred_Lasso_test2)
for name_metric, error in lasso_metrics2.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 4.281332398719396e-06

Train MSE: 0.02358
Train MAE: 0.09391
Train RMSE: 0.15356

Test MSE: 0.05824
Test MAE: 0.19457
Test RMSE: 0.24132


### ElasticNet Regression

In [22]:
# Creating and fitting model
ElasticNet_regressor2 = ElasticNet()
elasticnet_reg_parameters2 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_elasticnet2 = GridSearchCV(ElasticNet_regressor2, elasticnet_reg_parameters2, cv=5)

grid_search_cv_elasticnet2.fit(X_train2, y_train2)
grid_search_cv_elasticnet2.best_params_

# Model predictions
Elasticnet_best_reg2 = grid_search_cv_elasticnet2.best_estimator_
y_pred_Elasticnet_train2 = Elasticnet_best_reg2.predict(X_train2)
y_pred_Elasticnet_test2 = Elasticnet_best_reg2.predict(X_test2)

#The best Hyperparameters
elasticnet_best_key2 = list((grid_search_cv_elasticnet2.best_params_).keys())[0]
elasticnet_best_value2 = list((grid_search_cv_elasticnet2.best_params_).values())[0]
print(f'The best hyperparameters: {elasticnet_best_key2} = {elasticnet_best_value2}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train2, y_pred=y_pred_Elasticnet_train2).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
elasticnet_metrics2 = quality_metrics(y_true=y_test2, y_pred=y_pred_Elasticnet_test2)
for name_metric, error in elasticnet_metrics2.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 6.158482110660267e-05

Train MSE: 0.02359
Train MAE: 0.09396
Train RMSE: 0.15357

Test MSE: 0.05777
Test MAE: 0.19266
Test RMSE: 0.24036


### Random Forest

In [23]:
# Creating and fitting model
Forest_regressor2 = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)
forest_reg_parameters2 = {'max_depth': np.arange(4, 20, 2)}
grid_search_cv_forest2 = GridSearchCV(Forest_regressor2, forest_reg_parameters2, cv=3)
grid_search_cv_forest2.fit(X_train2, y_train2)
grid_search_cv_forest2.best_params_

# Model predictions
Forest_best_reg2 = grid_search_cv_forest2.best_estimator_
y_pred_Forest_train2 = Forest_best_reg2.predict(X_train2)
y_pred_Forest_test2 = Forest_best_reg2.predict(X_test2)

#The best Hyperparameters
forest_best_key2 = list((grid_search_cv_forest2.best_params_).keys())[0]
forest_best_value2 = list((grid_search_cv_forest2.best_params_).values())[0]
print(f'The best hyperparameters: {forest_best_key2} = {forest_best_value2}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train2, y_pred=y_pred_Forest_train2).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
rforest_metrics2 = quality_metrics(y_true=y_test2, y_pred=y_pred_Forest_test2)
for name_metric, error in rforest_metrics2.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: max_depth = 8

Train MSE: 0.01921
Train MAE: 0.06691
Train RMSE: 0.13859

Test MSE: 0.00854
Test MAE: 0.07438
Test RMSE: 0.09243


### k-NN Regression

In [24]:
# Creating and fitting model
KNeighbors_regressor2 = KNeighborsRegressor()
kNN_reg_parameters2 = {'n_neighbors': np.arange(4, 20, 2)}                  # <- np.arange(2, 6, 1)
grid_search_cv_kNN2 = GridSearchCV(KNeighbors_regressor2, kNN_reg_parameters2, cv=5)
grid_search_cv_kNN2.fit(X_train2, y_train2)
grid_search_cv_kNN2.best_params_

# Model predictions
kNN_best_reg2 = grid_search_cv_kNN2.best_estimator_
y_pred_kNN_train2 = kNN_best_reg2.predict(X_train2)
y_pred_kNN_test2 = kNN_best_reg2.predict(X_test2)

#The best Hyperparameters
kNN_best_key2 = list((grid_search_cv_kNN2.best_params_).keys())[0]
kNN_best_value2 = list((grid_search_cv_kNN2.best_params_).values())[0]
print(f'The best hyperparameters: {kNN_best_key2} = {kNN_best_value2}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train2, y_pred=y_pred_kNN_train2).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
knn_metrics2 = quality_metrics(y_true=y_test2, y_pred=y_pred_kNN_test2)
for name_metric, error in knn_metrics2.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: n_neighbors = 16

Train MSE: 0.04343
Train MAE: 0.11271
Train RMSE: 0.20840

Test MSE: 0.00965
Test MAE: 0.08614
Test RMSE: 0.09825


### Boosting

In [25]:
np.random.seed(42)
random.seed(42)

# Creating and fitting model
GBoosting_regressor2 = GradientBoostingRegressor(n_estimators=50, max_depth=18)
gboost_reg_parameters2 = {}
grid_search_cv_gboost2 = GridSearchCV(GBoosting_regressor2, gboost_reg_parameters2, cv=5)
grid_search_cv_gboost2.fit(X_train2, y_train2)
#print(grid_search_cv_gboost.best_params_)

# Model predictions
GBoost_best_reg2 = grid_search_cv_gboost2.best_estimator_
y_pred_gboost_train2 = GBoost_best_reg2.predict(X_train2)
y_pred_gboost_test2 = GBoost_best_reg2.predict(X_test2)

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train2, y_pred=y_pred_gboost_train2).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
gboost_metrics2 = quality_metrics(y_true=y_test2, y_pred=y_pred_gboost_test2)
for name_metric, error in gboost_metrics2.items():
    print(f'Test {name_metric}: {error:.5f}')

Train MSE: 0.01563
Train MAE: 0.03231
Train RMSE: 0.12502

Test MSE: 0.04187
Test MAE: 0.15225
Test RMSE: 0.20463


## Evaluation

In [26]:
fig = go.Figure()

model_name = ['Ridge', "Lasso","ElasticNet", 'RForest', 'kNN', 'Gradient Boosting']
model_metrics = [ridge_metrics2,lasso_metrics2, elasticnet_metrics2, rforest_metrics2, knn_metrics2, gboost_metrics2]

for name, metrics in zip(model_name, model_metrics):
    fig.add_trace(go.Histogram(histfunc= 'max',
                            x = list(metrics.keys()),
                                y = list(metrics.values()),
                                name = name,
                                ))

fig.update_layout(font_size = 25,
                    font_color='black',
                    title = "Model Evaluation",
                    plot_bgcolor = 'rgba(250,250,250,1)',
                    width = 1100,
                    height = 550,
                    # LEGEND
                    legend_title = "Models",
                    legend_font_size = 25,
                    legend_x = 1.02,
                    legend_y = 1,
                    #legend_bordercolor = 'black',
                    #legend_borderwidth = 1,
                    legend_itemsizing = 'trace',
                    legend_itemwidth=100,
                    # X-axis
                    xaxis_title = "WP6 + generated data (50%)",
                    xaxis_nticks = 7,
                    xaxis_ticklen = 16,
                    xaxis_tickwidth = 3,
                    xaxis_ticks = 'outside',
                    # Y-axis
                    yaxis_title = "Error",
                    yaxis_nticks = 10,
                    yaxis_ticklen = 16,
                    yaxis_tickwidth = 3,
                    yaxis_ticks = 'outside'
                    )

fig.add_shape(type="rect",
                xref="paper",
                yref="paper",
                x0=0,
                y0=0,
                x1=1.0,
                y1=1.0,
        line=dict(
            color="black",
                width=1,))
fig.layout.font.family = 'sans-serif'

fig.show()

> Better than the original WP6 dataset, but worse than previous obtained metrics

### Table of metrics

In [27]:
#Metrics table
data = []

for i in range(3):
    row = []
    for j in range(6):
        key = list(model_metrics[j].keys())[i]
        row.append(model_metrics[j][key])
    data.append(row)

WP6_50_metrics = pd.DataFrame(data, columns=model_name[:len(data[0])])
WP6_50_metrics['Metric'] = ['MSE', 'MAE', 'RMSE']
WP6_50_metrics['Dataset'] = ['WP6 + gen.data (50%)', 'WP6 + gen.data (50%)', 'WP6 + gen.data (50%)']
WP6_50_metrics["Linear"] = list(lin_reg_metrics1.values())

WP6_50_metrics = WP6_50_metrics[['Metric', 'Dataset', 'Linear'] + WP6_50_metrics.columns[:-3].tolist()]
WP6_50_metrics

,Metric,Dataset,Linear,Ridge,Lasso,ElasticNet,RForest,kNN,Gradient Boosting
0,MSE,WP6 + gen.data (50%),0.030822,0.057818,0.058236,0.057774,0.008543,0.009653,0.041874
1,MAE,WP6 + gen.data (50%),0.133292,0.192894,0.194575,0.192661,0.074378,0.086137,0.152255
2,RMSE,WP6 + gen.data (50%),0.175563,0.240453,0.241321,0.240361,0.092429,0.098251,0.204632


### Table of hyperparameters

In [28]:
#Best hyperparameters table
hyperparams_dict2 = {"Name of ML model": ["Linear Regression", "Ridge", "Lasso", "ElasticNet",
                                         "Random Forest", "kNN Regression", "XGBoost"],
                    "Tuning Hyperparameter": ["-", "Alpha", "Alpha", "Alpha", "Max Depth", "Number of Neighbors", "Number of Estimators, Max Depth"],
                    "Value": [None, ridge_best_value2, lasso_best_value2, elasticnet_best_value2, forest_best_value2,
                              kNN_best_value2, [50, 18]]}

best_hyperparams2 = pd.DataFrame(hyperparams_dict2)
best_hyperparams2

,Name of ML model,Tuning Hyperparameter,Value
0,Linear Regression,-,None
1,Ridge,Alpha,0.00336
2,Lasso,Alpha,0.000004
3,ElasticNet,Alpha,0.000062
4,Random Forest,Max Depth,8
5,kNN Regression,Number of Neighbors,16
6,XGBoost,"Number of Estimators, Max Depth","[50, 18]"


# Ratio of 31:21 (60%)

In [29]:
df = pd.concat([real_df, gen_df], ignore_index=True)
df.shape

(52, 13)

In [30]:
X3 = df.drop(['Ka_mean'], axis=1)
y3 = df["Ka_mean"]
print(f'General quantity of samples: {X3.shape[0]}')
print(f'Quantity of features: {X3.shape[1]}')

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.1, shuffle=False, random_state=42)
print(f"Length of training data: {len(X_train3)}")
print(f"Length of testing data: {len(X_test3)}")

General quantity of samples: 52
Quantity of features: 12
Length of training data: 46
Length of testing data: 6


## Building machine learning models

### Linear Regression

In [31]:
# Creating and fitting model
Lin_regressor3 = LinearRegression(n_jobs=-1)
grid_search_cv_linear3 = GridSearchCV(Lin_regressor3, {}, cv=5)
grid_search_cv_linear3.fit(X_train3, y_train3.to_numpy())

# Model predictions
y_pred_Linear_train3 = grid_search_cv_linear3.best_estimator_.predict(X_train3)
y_pred_Linear_test3 = grid_search_cv_linear3.best_estimator_.predict(X_test3)

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train3, y_pred=y_pred_Linear_train3).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
lin_reg_metrics3 = quality_metrics(y_true=y_test3, y_pred=y_pred_Linear_test3)
for name_metric, error in lin_reg_metrics3.items():
    print(f'Test {name_metric}: {error:.5f}')

Train MSE: 0.02538
Train MAE: 0.10486
Train RMSE: 0.15932

Test MSE: 0.01883
Test MAE: 0.11394
Test RMSE: 0.13723


### Ridge Regression

In [32]:
# Creating and fitting model
Ridge_regressor3 = Ridge()
ridge_reg_parameters3 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_ridge3 = GridSearchCV(Ridge_regressor3, ridge_reg_parameters3, cv=5)

grid_search_cv_ridge3.fit(X_train3, y_train3)
grid_search_cv_ridge3.best_params_

# Model predictions
Ridge_best_reg3 = grid_search_cv_ridge3.best_estimator_
y_pred_Ridge_train3 = Ridge_best_reg3.predict(X_train3)
y_pred_Ridge_test3 = Ridge_best_reg3.predict(X_test3)

#The best Hyperparameters
ridge_best_key3 = list((grid_search_cv_ridge3.best_params_).keys())[0]
ridge_best_value3 = list((grid_search_cv_ridge3.best_params_).values())[0]
print(f'The best hyperparameters: {ridge_best_key3} = {ridge_best_value3}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train3, y_pred=y_pred_Ridge_train3).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
ridge_metrics3 = quality_metrics(y_true=y_test3, y_pred=y_pred_Ridge_test3)
for name_metric, error in ridge_metrics3.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 10.0

Train MSE: 0.03112
Train MAE: 0.10912
Train RMSE: 0.17640

Test MSE: 0.00651
Test MAE: 0.06636
Test RMSE: 0.08071


### Lasso Regresssion

In [33]:
# Creating and fitting model
Lasso_regressor3 = Lasso()
lasso_reg_parameters3 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_lasso3 = GridSearchCV(Lasso_regressor3, lasso_reg_parameters3, cv=5)

grid_search_cv_lasso3.fit(X_train3, y_train3)
grid_search_cv_lasso3.best_params_

# Model predictions
Lasso_best_reg3 = grid_search_cv_lasso3.best_estimator_
y_pred_Lasso_train3 = Lasso_best_reg3.predict(X_train3)
y_pred_Lasso_test3 = Lasso_best_reg3.predict(X_test3)

#The best Hyperparameters
lasso_best_key3 = list((grid_search_cv_lasso3.best_params_).keys())[0]
lasso_best_value3 = list((grid_search_cv_lasso3.best_params_).values())[0]
print(f'The best hyperparameters: {lasso_best_key3} = {lasso_best_value3}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train3, y_pred=y_pred_Lasso_train3).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
lasso_metrics3 = quality_metrics(y_true=y_test3, y_pred=y_pred_Lasso_test3)
for name_metric, error in lasso_metrics3.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 0.18329807108324375

Train MSE: 0.04659
Train MAE: 0.12281
Train RMSE: 0.21585

Test MSE: 0.00321
Test MAE: 0.04814
Test RMSE: 0.05667


### ElasticNet Regression

In [34]:
# Creating and fitting model
ElasticNet_regressor3 = ElasticNet()
elasticnet_reg_parameters3 = {'alpha': np.logspace(-10, 1, 20)}
grid_search_cv_elasticnet3 = GridSearchCV(ElasticNet_regressor3, elasticnet_reg_parameters3, cv=5)

grid_search_cv_elasticnet3.fit(X_train3, y_train3)
grid_search_cv_elasticnet3.best_params_

# Model predictions
Elasticnet_best_reg3 = grid_search_cv_elasticnet3.best_estimator_
y_pred_Elasticnet_train3 = Elasticnet_best_reg3.predict(X_train3)
y_pred_Elasticnet_test3 = Elasticnet_best_reg3.predict(X_test3)

#The best Hyperparameters
elasticnet_best_key3 = list((grid_search_cv_elasticnet3.best_params_).keys())[0]
elasticnet_best_value3 = list((grid_search_cv_elasticnet3.best_params_).values())[0]
print(f'The best hyperparameters: {elasticnet_best_key3} = {elasticnet_best_value3}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train3, y_pred=y_pred_Elasticnet_train3).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
elasticnet_metrics3 = quality_metrics(y_true=y_test3, y_pred=y_pred_Elasticnet_test3)
for name_metric, error in elasticnet_metrics3.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: alpha = 0.18329807108324375

Train MSE: 0.04659
Train MAE: 0.12281
Train RMSE: 0.21585

Test MSE: 0.00321
Test MAE: 0.04814
Test RMSE: 0.05667


### Random Forest

In [35]:
# Creating and fitting model
Forest_regressor3 = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)
forest_reg_parameters3 = {'max_depth': np.arange(4, 20, 2)}
grid_search_cv_forest3 = GridSearchCV(Forest_regressor3, forest_reg_parameters3, cv=3)
grid_search_cv_forest3.fit(X_train3, y_train3)
grid_search_cv_forest3.best_params_

# Model predictions
Forest_best_reg3 = grid_search_cv_forest3.best_estimator_
y_pred_Forest_train3 = Forest_best_reg3.predict(X_train3)
y_pred_Forest_test3 = Forest_best_reg3.predict(X_test3)

#The best Hyperparameters
forest_best_key3 = list((grid_search_cv_forest3.best_params_).keys())[0]
forest_best_value3 = list((grid_search_cv_forest3.best_params_).values())[0]
print(f'The best hyperparameters: {forest_best_key3} = {forest_best_value3}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train3, y_pred=y_pred_Forest_train3).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
rforest_metrics3 = quality_metrics(y_true=y_test3, y_pred=y_pred_Forest_test3)
for name_metric, error in rforest_metrics3.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: max_depth = 6

Train MSE: 0.01416
Train MAE: 0.05724
Train RMSE: 0.11900

Test MSE: 0.02456
Test MAE: 0.09548
Test RMSE: 0.15672


### k-NN Regression

In [36]:
# Creating and fitting model
KNeighbors_regressor3 = KNeighborsRegressor()
kNN_reg_parameters3 = {'n_neighbors': np.arange(4, 20, 2)}                  # <- np.arange(2, 6, 1)
grid_search_cv_kNN3 = GridSearchCV(KNeighbors_regressor3, kNN_reg_parameters3, cv=5)
grid_search_cv_kNN3.fit(X_train3, y_train3)
grid_search_cv_kNN3.best_params_

# Model predictions
kNN_best_reg3 = grid_search_cv_kNN3.best_estimator_
y_pred_kNN_train3 = kNN_best_reg3.predict(X_train3)
y_pred_kNN_test3 = kNN_best_reg3.predict(X_test3)

#The best Hyperparameters
kNN_best_key3 = list((grid_search_cv_kNN3.best_params_).keys())[0]
kNN_best_value3 = list((grid_search_cv_kNN3.best_params_).values())[0]
print(f'The best hyperparameters: {kNN_best_key3} = {kNN_best_value3}')
print()

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train3, y_pred=y_pred_kNN_train3).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
knn_metrics3 = quality_metrics(y_true=y_test3, y_pred=y_pred_kNN_test3)
for name_metric, error in knn_metrics3.items():
    print(f'Test {name_metric}: {error:.5f}')

The best hyperparameters: n_neighbors = 18

Train MSE: 0.03686
Train MAE: 0.11120
Train RMSE: 0.19198

Test MSE: 0.00838
Test MAE: 0.08044
Test RMSE: 0.09156


### Boosting

In [37]:
np.random.seed(42)
random.seed(42)

# Creating and fitting model
GBoosting_regressor3 = GradientBoostingRegressor(n_estimators=50, max_depth=18)
gboost_reg_parameters3 = {}
grid_search_cv_gboost3 = GridSearchCV(GBoosting_regressor3, gboost_reg_parameters3, cv=5)
grid_search_cv_gboost3.fit(X_train3, y_train3)
#print(grid_search_cv_gboost.best_params_)

# Model predictions
GBoost_best_reg3 = grid_search_cv_gboost3.best_estimator_
y_pred_gboost_train3 = GBoost_best_reg3.predict(X_train3)
y_pred_gboost_test3 = GBoost_best_reg3.predict(X_test3)

# Calculating metrics
for name_metric, error in quality_metrics(y_true=y_train3, y_pred=y_pred_gboost_train3).items():
    print(f'Train {name_metric}: {error:.5f}')

print()
gboost_metrics3 = quality_metrics(y_true=y_test3, y_pred=y_pred_gboost_test3)
for name_metric, error in gboost_metrics3.items():
    print(f'Test {name_metric}: {error:.5f}')

Train MSE: 0.01053
Train MAE: 0.02192
Train RMSE: 0.10264

Test MSE: 0.02609
Test MAE: 0.09203
Test RMSE: 0.16153


## Evaluation

In [38]:
fig = go.Figure()

model_name = ['Ridge', "Lasso","ElasticNet", 'RForest', 'kNN', 'Gradient Boosting']
model_metrics = [ridge_metrics3,lasso_metrics3, elasticnet_metrics3, rforest_metrics3, knn_metrics3, gboost_metrics3]

for name, metrics in zip(model_name, model_metrics):
    fig.add_trace(go.Histogram(histfunc= 'max',
                            x = list(metrics.keys()),
                                y = list(metrics.values()),
                                name = name,
                                ))

fig.update_layout(font_size = 25,
                    font_color='black',
                    title = "Model Evaluation",
                    plot_bgcolor = 'rgba(250,250,250,1)',
                    width = 1100,
                    height = 550,
                    # LEGEND
                    legend_title = "Models",
                    legend_font_size = 25,
                    legend_x = 1.02,
                    legend_y = 1,
                    #legend_bordercolor = 'black',
                    #legend_borderwidth = 1,
                    legend_itemsizing = 'trace',
                    legend_itemwidth=100,
                    # X-axis
                    xaxis_title = "WP6 + generated data (60%)",
                    xaxis_nticks = 7,
                    xaxis_ticklen = 16,
                    xaxis_tickwidth = 3,
                    xaxis_ticks = 'outside',
                    # Y-axis
                    yaxis_title = "Error",
                    yaxis_nticks = 10,
                    yaxis_ticklen = 16,
                    yaxis_tickwidth = 3,
                    yaxis_ticks = 'outside'
                    )

fig.add_shape(type="rect",
                xref="paper",
                yref="paper",
                x0=0,
                y0=0,
                x1=1.0,
                y1=1.0,
        line=dict(
            color="black",
                width=1,))
fig.layout.font.family = 'sans-serif'

fig.show()

> Metrics are good, but there is also a risk of smearing the data as the quantity of synthetic samples is greater than the quantity of original samples.

### Table of metrics

In [39]:
#Metrics table
data = []

for i in range(3):
    row = []
    for j in range(6):
        key = list(model_metrics[j].keys())[i]
        row.append(model_metrics[j][key])
    data.append(row)

WP6_60_metrics = pd.DataFrame(data, columns=model_name[:len(data[0])])
WP6_60_metrics['Metric'] = ['MSE', 'MAE', 'RMSE']
WP6_60_metrics['Dataset'] = ['WP6 + gen.data (60%)', 'WP6 + gen.data (60%)', 'WP6 + gen.data (60%)']
WP6_60_metrics["Linear"] = list(lin_reg_metrics3.values())

WP6_60_metrics = WP6_60_metrics[['Metric', 'Dataset', 'Linear'] + WP6_60_metrics.columns[:-3].tolist()]
WP6_60_metrics

,Metric,Dataset,Linear,Ridge,Lasso,ElasticNet,RForest,kNN,Gradient Boosting
0,MSE,WP6 + gen.data (60%),0.018833,0.006514,0.003211,0.003211,0.024561,0.008383,0.026092
1,MAE,WP6 + gen.data (60%),0.113940,0.066358,0.048143,0.048143,0.095477,0.080436,0.092030
2,RMSE,WP6 + gen.data (60%),0.137234,0.080710,0.056667,0.056667,0.156718,0.091558,0.161531


### Table of hyperparameters

In [40]:
#Best hyperparameters table
hyperparams_dict3 = {"Name of ML model": ["Linear Regression", "Ridge", "Lasso", "ElasticNet",
                                         "Random Forest", "kNN Regression", "XGBoost"],
                    "Tuning Hyperparameter": ["-", "Alpha", "Alpha", "Alpha", "Max Depth", "Number of Neighbors", "Number of Estimators, Max Depth"],
                    "Value": [None, ridge_best_value3, lasso_best_value3, elasticnet_best_value3, forest_best_value3,
                              kNN_best_value3, [50, 18]]}

best_hyperparams3 = pd.DataFrame(hyperparams_dict3)
best_hyperparams3

,Name of ML model,Tuning Hyperparameter,Value
0,Linear Regression,-,None
1,Ridge,Alpha,10.0
2,Lasso,Alpha,0.183298
3,ElasticNet,Alpha,0.183298
4,Random Forest,Max Depth,6
5,kNN Regression,Number of Neighbors,18
6,XGBoost,"Number of Estimators, Max Depth","[50, 18]"


# General Tables

In [41]:
General_WP6_gen = pd.concat([WP6_25_metrics, WP6_50_metrics, WP6_60_metrics], ignore_index=True)
General_WP6_gen.to_csv("METRICS_WP6_gen.csv")
General_WP6_gen

,Metric,Dataset,Linear,Ridge,Lasso,ElasticNet,RForest,kNN,Gradient Boosting
0,MSE,WP6 + gen.data (25%),0.030822,0.004933,0.005351,0.003981,0.001261,0.001498,0.001852
1,MAE,WP6 + gen.data (25%),0.133292,0.049281,0.068924,0.053907,0.030526,0.035074,0.032566
2,RMSE,WP6 + gen.data (25%),0.175563,0.070236,0.073148,0.063093,0.035517,0.038702,0.043029
3,MSE,WP6 + gen.data (50%),0.030822,0.057818,0.058236,0.057774,0.008543,0.009653,0.041874
4,MAE,WP6 + gen.data (50%),0.133292,0.192894,0.194575,0.192661,0.074378,0.086137,0.152255
5,RMSE,WP6 + gen.data (50%),0.175563,0.240453,0.241321,0.240361,0.092429,0.098251,0.204632
6,MSE,WP6 + gen.data (60%),0.018833,0.006514,0.003211,0.003211,0.024561,0.008383,0.026092
7,MAE,WP6 + gen.data (60%),0.113940,0.066358,0.048143,0.048143,0.095477,0.080436,0.092030
8,RMSE,WP6 + gen.data (60%),0.137234,0.080710,0.056667,0.056667,0.156718,0.091558,0.161531
